In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.model_selection import train_test_split

print(tf.__version__)
print(keras.__version__)

1.12.0
2.1.6-tf


In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
import tensorflow.keras.applications.resnet50 as resnet50 
import tensorflow.keras.applications.vgg19 as vgg19
import tensorflow.keras.applications.densenet as densenet
import tensorflow.keras.applications.inception_v3 as inception_v3

In [4]:
import tensorflow.keras.preprocessing.image as image
import tensorflow.keras.losses as losses
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.callbacks as callbacks

# Dataset

In [5]:
from scipy.io import loadmat
from os.path import join

prefix = '../../Stanford_Dog_Breed'
imgfix = 'Images'
anofix = 'Annotation'

In [ ]:
# train = loadmat(join(prefix, 'train_list.mat'))
# test = loadmat(join(prefix, 'test_list.mat'))

# L_annotation = L['annotation_list'];
# L_file = L['file_list'];
# L = L['labels'];

# ftrain = train['file_list']
# ftest = test['file_list']

# ltrain = train['labels']
# ltest = test['labels']

# ntrain = ltrain.shape[0]
# ntest = ltest.shape[0]

# print('Size of the training set:\t', ntrain)
# print 'Size of the testing set: \t', ntest

In [6]:
import tensorflow.keras.preprocessing.image as image

ImgGen = image.ImageDataGenerator(preprocessing_function=inception_v3.preprocess_input, 
                                  width_shift_range=0.2, 
                                  height_shift_range=0.2, 
                                  shear_range=0.2, 
                                  zoom_range=0.2, 
                                  horizontal_flip=True, 
                                  validation_split=1/6.)

Train = ImgGen.flow_from_directory(join(prefix, imgfix, 'Train'), 
                                   target_size=(224, 224), 
                                   class_mode='categorical', 
                                   batch_size=32, 
                                   shuffle=True, 
                                   seed=None, 
                                   subset='training', 
                                   interpolation='nearest')

Valid = ImgGen.flow_from_directory(join(prefix, imgfix, 'Train'), 
                                   target_size=(224, 224), 
                                   class_mode='categorical', 
                                   batch_size=32, 
                                   shuffle=True, 
                                   seed=None, 
                                   subset='validation', 
                                   interpolation='nearest')


ImgTestGen = image.ImageDataGenerator(preprocessing_function=inception_v3.preprocess_input, validation_split=1/6.)

Test  = ImgTestGen.flow_from_directory(join(prefix, imgfix, 'Test' ), 
                                       target_size=(224, 224), 
                                       class_mode='categorical', 
                                       batch_size=32, 
                                       shuffle=False, 
                                       interpolation='nearest')







Found 10080 images belonging to 120 classes.
Found 1920 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [ ]:
Test.class_indices

## Catagory 2 Index Table

DogBreed = {}
for i in range(1, ntest):
    l = ltest[i][0]
    c = ftest[i][0][0].split('/')[0]
    c = c.split('-')[1]
    if c not in DogBreed:
        DogBreed[c] = l - 1
        print c, l - 1


def gen_path(idx):
    try:    path = join(prefix, imgfix, ftest[idx][0][0])
    except: path = join(prefix, imgfix, ftest[-1][0][0])      
    return  path

def gen_labl(idx):
    try:    labl = ltest[idx][0]
    except: labl = ltest[-1][0]
    return  labl

def get_imgs(idx):
    img = image.load_img(gen_path(idx), target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    return x

# Models

## Data Preparation

XTrain = []
for idx in range(ntrain):
    path = join(prefix, imgfix, ftrain[idx][0][0])
    try:
        img = image.load_img(path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        XTrain.append(resnet50.preprocess_input(x))
    except:
        print('Error at processing the', idx, 'images.')

XTrain = np.vstack(XTrain)

XTest = []
for idx in range(ntest):
    path = gen_path(idx)
    try:
        img = image.load_img(path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        XTest.append(resnet50.preprocess_input(x))
    except:
        print('Error at processing the', idx, 'images.')
        
XTest = np.vstack(XTest)

# print(XTrain.shape)
print(XTest.shape)

## Classify using ImageNet Classes with ResNet50

In [ ]:
resnet50_naive = resnet50.ResNet50(weights='imagenet')
# YTest = resnet50_naive.predict(XTest)

In [ ]:
resnet50_naive.compile('SGD', loss=losses.categorical_crossentropy, metrics={'output_a': 'accuracy'})

In [ ]:
YTest_ = resnet50_naive.evaluate_generator(Test)

In [ ]:
YTest_

In [ ]:
Test.total_batches_seen

In [ ]:
print XTest[0:1000, :, :, :].shape
print YTest.shape

In [ ]:
# YTest = resnet50.decode_predictions(YTest, top=3)[0]

correct, top = 0, 1

for idx in range(ntest):
    pred = resnet50.decode_predictions(YTest[idx:idx+1,:], top=top)[0]
    for k in range(top):
        if pred[k][1] not in DogBreed:
            continue
        labl = DogBreed[pred[k][1]]
        if labl == gen_labl(idx):
            correct += 1
            break


print(correct)
print(correct / float(ntest))
    

## VGG19

In [ ]:
vgg19_naive = vgg19.VGG19(weights='imagenet')
vgg19_model = keras.models.Model(inputs=vgg19_naive.input, 
                                 outputs=vgg19_naive.get_layer('block4_pool').output)

In [ ]:
vgg19_naive.get_layer(name=0, index=20)

In [ ]:
# XTest = []
for idx in range(ntest):
    path = gen_path(idx)
    try:
        x = get_imgs(idx)
        XTest.append(vgg19.preprocess_input(x))
    except:
        print('Error at processing the', idx, 'images.')
        
XTest = np.vstack(XTest)

In [ ]:
YTest = vgg19_naive.predict(XTest)

X1 = vgg19.preprocess_input(x)
X2 = resnet50.preprocess_input(x)
X3 = densenet.preprocess_input(x)
X4 = inception_v3.preprocess_input(x)

print np.sum(X2 - X1)
print np.sum(X3 - X2)
print np.sum(X4 - X3)

In [ ]:
correct, top = 0, 1

for idx in range(ntest):
    pred = vgg19.decode_predictions(YTest[idx:idx+1,:], top=top)[0]
    for k in range(top):
        if pred[k][1] not in DogBreed:
            continue
        labl = DogBreed[pred[k][1]]
        if labl == gen_labl(idx):
            correct += 1
            break


print(correct)
print(correct / float(ntest))

## DenseNet

In [ ]:
densenet121_naive = densenet.DenseNet121(weights='imagenet', include_top=False)

In [ ]:
densenet121_naive.summary()

In [ ]:
densenet121_naive.summary()

In [ ]:
XTest = []
for idx in range(ntest):
    path = gen_path(idx)
    try:
        x = get_imgs(idx)
        XTest.append(densenet.preprocess_input(x))
    except:
        print('Error at processing the', idx, 'images.')
        
XTest = np.vstack(XTest)

In [ ]:
YTest = densenet121_naive.predict(XTest)

In [ ]:
correct, top = 0, 1

for idx in range(ntest):
    pred = densenet.decode_predictions(YTest[idx:idx+1,:], top=top)[0]
    for k in range(top):
        if pred[k][1] not in DogBreed:
            continue
        labl = DogBreed[pred[k][1]]
        if labl == gen_labl(idx):
            correct += 1
            break


print(correct)
print(correct / float(ntest))

## InceptionV3

In [ ]:
class SaveModelandEval(callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch == 0: return
        if epoch % 20 == 0:
            models.save_model(inceptionv3_modified, 'fc120'+str(epoch) )
            print inceptionv3_modified.evaluate_generator(Test,  verbose=1) 
            

In [ ]:
inceptionv3_naive = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [ ]:
x = inceptionv3_naive.output
print x.shape

# x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
# print x.shape

x = layers.GlobalAveragePooling2D()(x)
print x.shape

# x = layers.Dense(1024, activation='relu')(x)
# print x.shape
pred = layers.Dense(120, activation='softmax')(x)
print pred.shape

In [ ]:
inceptionv3_modified = models.Model(inputs=inceptionv3_naive.input, outputs=pred)

for layer in inceptionv3_naive.layers:
    layer.trainable = False

inceptionv3_modified.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
# 

In [ ]:
inceptionv3_modified = models.load_model('fc12080')

In [ ]:
inceptionv3_modified.summary()

In [ ]:
print inceptionv3_modified.evaluate_generator(Train, verbose=1)
print inceptionv3_modified.evaluate_generator(Valid, verbose=1)
print inceptionv3_modified.evaluate_generator(Test,  verbose=1)

In [ ]:
inceptionv3_modified.fit_generator(
    Train, 
#     steps_per_epoch=5, 
    epochs=121, 
    verbose=1, 
#     validation_data=Valid, 
#     validation_steps=60, 
    initial_epoch=100,
    callbacks=[SaveModelandEval()])

In [ ]:
models.save_model(inceptionv3_modified, 'fc120')

## 20 epochs

In [ ]:
print inceptionv3_modified.evaluate_generator(Train, verbose=1)
print inceptionv3_modified.evaluate_generator(Valid, verbose=1)
print inceptionv3_modified.evaluate_generator(Test, verbose=1)

## 40 epochs

In [ ]:
print inceptionv3_modified.evaluate_generator(Train, verbose=1)
print inceptionv3_modified.evaluate_generator(Valid, verbose=1)
print inceptionv3_modified.evaluate_generator(Test, verbose=1)

### Save the trained models

In [ ]:
models.save_model(incptionv3_modified, 'fc120')

In [ ]:
inceptionv3_modified.metrics_names



In [ ]:
incptionv3_modified.fit_generator(Train, steps_per_epoch=100, epochs=10, verbose=1)

In [ ]:
Eval_35 = incptionv3_modified.evaluate_generator(Test)
print(Eval_35)

In [ ]:
h = incptionv3_modified.fit_generator(Train, epochs=10, verbose=1)

In [ ]:
Eval_45 = incptionv3_modified.evaluate_generator(Test)
print(Eval_45)

In [ ]:
Eval_55 = incptionv3_modified.evaluate_generator(Test)
print(Eval_55)

XTest = []
for idx in range(ntest):
    path = gen_path(idx)
    try:
        x = get_imgs(idx)
        XTest.append(inception_v3.preprocess_input(x))
    except:
        print('Error at processing the', idx, 'images.')
        
XTest = np.vstack(XTest)

YTest = inceptionv3_naive.predict(XTest)

correct, top = 0, 1

for idx in range(ntest):
    pred = inception_v3.decode_predictions(YTest[idx:idx+1,:], top=top)[0]
    for k in range(top):
        if pred[k][1] not in DogBreed:
            continue
        labl = DogBreed[pred[k][1]]
        if labl == gen_labl(idx):
            correct += 1
            break

print(correct)
print(correct / float(ntest))